In [83]:
import subprocess
import requests
import json
import time

STELLARIUM_EXE = "C:\Program Files\Stellarium\stellarium.exe"

#C:\Users\mered\AppData\Roaming\Stellarium

# https://stellarium.org/doc/0.18/
# http://stellarium.sourceforge.net/wiki/index.php/Advanced_Use#Command_Line_Options
# https://astronomy.stackexchange.com/questions/35508/convert-stellarium-video-capture-to-360-for-youtube


In [98]:
from pathlib import Path

#config_path = Path("./stars.ini")

p = subprocess.Popen([STELLARIUM_EXE, 
    "-c", "./stars.ini",
    "--spout",
    "--spout-name", "stellarium"
])


# p = subprocess.Popen([STELLARIUM_EXE, 
#     "-c", "./default.ini",
#     "--spout",
#     "--spout-name", "stellarium"
# ])




In [101]:

from datetime import datetime
import time
import julian
from math import pow

# screen_h                               = 1440
# screen_number                          = 1
# screen_w                               = 2560


def lmap( value, start1, stop1, start2, stop2):
    return start2 + (stop2 - start2) * ((value - start1) / (stop1 - start1))

def print_json(data):
    print(json.dumps(data, indent=4, sort_keys=True))

def rate_to_JDay_per_sec(rate):
    return rate / 8640

def check_ret(ret):
    if ret.status_code == requests.codes.ok:
        return ret
    else:
        print(f"Request returned {ret.status_code}")
        return None

def hide_ui():
    print("Hide UI")
    return ( check_ret( requests.post("http://localhost:8090/api/stelaction/do", {"id" : "actionToggle_GuiHidden_Global" })),
            check_ret( requests.post("http://localhost:8090/api/stelaction/do", {"id" : "actionShow_Planets_Labels" })) )

def set_time_jd( jdtime, timerate):
    data = {
        "time" : jdtime,
        "timerate" : rate_to_JDay_per_sec( timerate )
    }
    print(f"Setting time\n{data}")
    return check_ret( requests.post("http://localhost:8090/api/main/time", data, timeout=0.1) )

def set_time( time, timerate):
    data = {
        "time" : julian.to_jd( datetime.strptime(time, '%b %d %Y %I:%M%p') ) ,
        "timerate" : rate_to_JDay_per_sec( timerate )
    }
    print(f"Setting time\n{data}")
    return check_ret( requests.post("http://localhost:8090/api/main/time", data, timeout=0.1) )

def get_status():
    print(f"Getting status")
    ret = check_ret( requests.get('http://localhost:8090/api/main/status', timeout=0.1) )
    if( ret ):
        return ret.json()
    return None

def connect():
    connecting = True
    print("Connecting...")
    while connecting:
        try:
            status = get_status()
            if status:
                print("Connected!")
                hide_ui()
                connecting = False
            else:
                r.raise_for_status()
        except requests.exceptions.Timeout as e:
            time.sleep(0.5)

# r = requests.get("http://localhost:8090/api/stelproperty/list", timeout=0.1)
# print(json.dumps(r.json(), indent=4, sort_keys=True))

# r = requests.get("http://localhost:8090/api/stelaction/list", timeout=0.1)
# print(json.dumps(r.json(), indent=4, sort_keys=True))

connect()

#-----------------



Connecting...
Getting status
Connected!
Hide UI
Setting time
{'time': 2459178.2708333335, 'timerate': 0.00011574074074074075}


<Response [200]>

In [ ]:
set_time('Nov 24 2020  6:30PM', 1.0)
time.sleep(60)

T  = 60
dt = 1.5
t = 0.0
while( t <= T):
    status = get_status()
    #print_json(status)
    jdtime = status["time"]["jday"]

    f = t / T
    rate = lmap(pow(f, 10.0), 0.0, 1.0, 1.0, 6000)

    print(f"t: {t:0.4f} rate: {rate:0.4f}" )
    set_time_jd(jdtime, rate)

    time.sleep(dt)
    t += dt


In [91]:
# p.terminate()
# returncode = p.wait()